In [1]:
%load_ext autoreload
%autoreload 2
# Import and set up some basic plotting parameters
from imports_paper_plots import *
set_context_and_font()

plt.rcParams["axes.facecolor"] = "1."

In [2]:
results_dir_path = Path(os.environ["DATAICNS"]) / "bioRNN/memory-capacity"
figs_path = results_dir_path / "figs"

### Don't forget to concatenate results again!

In [3]:
neuron_density = 1
results_hyperparams_path = results_dir_path/"hyperparams"/f"neuron-density-{neuron_density}"/"hyperparams_all"

results = pd.read_csv(results_hyperparams_path)
results.head(3)

,connectome,rand_partition,neuron_density,W,bias,input_scaling,leak_rate,n_transient,spectral_radius,mean_test_memory_capacity
0,macaque,True,1,bio_rank,1,0.000001,0.6,100,0.91,6.131167
1,macaque,True,1,bio_rank,1,0.000001,0.6,100,0.93,6.148464
2,macaque,True,1,bio_rank,1,0.000001,0.6,100,0.95,6.287349


In [4]:
# Drop columns that do not vary
results = results.drop(columns=["n_transient", "bias"])
results = results.rename(columns={"mean_test_memory_capacity": "memory_capacity"})
results.head(3)

,connectome,rand_partition,neuron_density,W,input_scaling,leak_rate,spectral_radius,memory_capacity
0,macaque,True,1,bio_rank,0.000001,0.6,0.91,6.131167
1,macaque,True,1,bio_rank,0.000001,0.6,0.93,6.148464
2,macaque,True,1,bio_rank,0.000001,0.6,0.95,6.287349


In [5]:
(results.connectome.unique(),
results.rand_partition.unique(),
results.input_scaling.unique(),)

(array(['macaque', 'marmoset', 'human'], dtype=object),
 array([ True, False]),
 array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00]))

In [6]:
results = refactor_names(
    df=results,
    capitalize_cols=["connectome"],
    to_rename=None
)
results.head(3)

,connectome,rand_partition,neuron_density,W,input_scaling,leak_rate,spectral_radius,memory_capacity
0,Macaque,True,1,Bio (rank),0.000001,0.6,0.91,6.131167
1,Macaque,True,1,Bio (rank),0.000001,0.6,0.93,6.148464
2,Macaque,True,1,Bio (rank),0.000001,0.6,0.95,6.287349


In [33]:
%%time
connectomes = ["Macaque", "Human", "Marmoset"]
rand_partitions = [True, False,]
for connectome, rand_partition in product(connectomes, rand_partitions):
    data = results[(results.connectome == connectome) &
                   (results.rand_partition == rand_partition)]

    g = plot.hyperparams_grid(
        data=data,
        x="spectral_radius",
        y="memory_capacity",
        palette=my_palette,
        #facet_kws={"ylim": (-2.5, 1)}
    )
    
    g.set_axis_labels(y_var="Memory capacity", x_var=r"Spectral radius ($\rho$)",)
    set_facetgrid_titles(
        g,
        col_template=r'Input scaling ($\epsilon$) = {col_name}',
        row_template=r"Leak rate ($\alpha$) = {row_name}", 
        size=23
    )
    
    g.fig.subplots_adjust(wspace=0.1, hspace=.003)
    plt.suptitle(f"{connectome}", x=.45, y=1., fontsize=40)
    g.savefig(
        figs_path
        /f"hyperparams/neuron-density-{neuron_density}"
        /f"{connectome}_rand-part-{rand_partition}"
    )
    plt.close()

CPU times: user 57.9 s, sys: 896 ms, total: 58.8 s
Wall time: 57.8 s
